<a href="https://colab.research.google.com/github/anaguilarar/agwise_data_sourcing/blob/main/GEEMODIS_data_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### INITIAL configuration

configuration = {
    'GENERAL_SETTINGS':{
      'ee_project_name': 'ee-anaguilarar'

      },
    'PREPROCESSING':{
        'crop_mask': True,
        'crop_mask_product': 'ESA'
    },
    'DATA_DOWNLOAD':
     {
      'ADM0_NAME': 'Kenya',
      'ADM1_NAME': 'Coast',
      'ADM2_NAME': None,
      'product': 'MOD13Q1',
      'starting_date': '2023-01-01',
      'ending_date': '2023-12-01',

    }
}


In [ ]:
import os

if not os.path.exists('agwise_data_sourcing'):
  !git clone https://github.com/anaguilarar/agwise_data_sourcing.git
  os.chdir('/content/agwise_data_sourcing')
else:
  os.chdir('/content/agwise_data_sourcing')


In [ ]:
from gee_datasets.gee_data import GEEMODIS
import ee
ee.Initialize(project=configuration['GENERAL_SETTINGS']['ee_project_name'])


In [ ]:
data_downloader = GEEMODIS(configuration['DATA_DOWNLOAD']['ADM0_NAME'], configuration['DATA_DOWNLOAD']['product'])
data_downloader.initialize_query(configuration['DATA_DOWNLOAD']['starting_date'], configuration['DATA_DOWNLOAD']['ending_date'])


In [ ]:
data_downloader.plot_time_series('NDVI')

In [ ]:
from gee_datasets.processing_funs import fill_gaps_linear, smooth_ts_using_savitsky_golay_modis, summarize_collection_tots
import matplotlib.pyplot as plt

data_filled = fill_gaps_linear(data_downloader.query, 'NDVI')

print("First smoothed image properties:",
      ee.Image(data_filled.first()).getInfo()["properties"])



In [ ]:
band = 'NDVI'
result = smooth_ts_using_savitsky_golay_modis(data_filled.select('NDVI'), window_size = 5)
print("First smoothed image properties:",
      ee.Image(result.first()).getInfo()["properties"])


In [ ]:
from utils.plots import plot_time_series_from_ic

band2 = 'NDVI_smooth'

df = plot_time_series_from_ic(
    ic=ee.ImageCollection(result),
    band=band2,
    summarize_fn=summarize_collection_tots,
    region_filter=data_downloader.country_filter,
    title=f"{band2} smoothed time series"
)


## CROP MASK

In [ ]:
from gee_datasets.gee_data import GEECropMask, GEEMODIS


data_downloader = GEEMODIS(configuration['DATA_DOWNLOAD']['ADM0_NAME'], configuration['DATA_DOWNLOAD']['product'])
data_downloader.initialize_query(configuration['DATA_DOWNLOAD']['starting_date'], configuration['DATA_DOWNLOAD']['ending_date'])


cropmask_downloader = GEECropMask(configuration['DATA_DOWNLOAD']['ADM0_NAME'], configuration['PREPROCESSING']['crop_mask_product'])
cropmask_downloader.initialize_query()

crop_mask = ee.Image(cropmask_downloader.query.first()).clip(cropmask_downloader.country_filter).eq(cropmask_downloader.crop_mask_value[cropmask_downloader.product])



In [ ]:
data_downloader.country_filter

dataset = ee.FeatureCollection(data_downloader._global_adiminstrative_data.format(adm_level='ADM1'))
dataset.filter(ee.Filter.eq('shapeName', 'Busia'))


In [ ]:
from gee_datasets.processing_funs import fill_gaps_linear, smooth_ts_using_savitsky_golay_modis

data_filled = fill_gaps_linear(data_downloader.query, 'NDVI')


datamasked = data_downloader.query.first().clip(cropmask_downloader.country_filter).updateMask(crop_mask)

data_filled_masked = data_filled.first().clip(cropmask_downloader.country_filter).updateMask(crop_mask)

data_filled_SG = smooth_ts_using_savitsky_golay_modis(data_filled.select('NDVI'), window_size = 5)


data_filled_SG_masked = data_filled_SG.first().clip(cropmask_downloader.country_filter).updateMask(crop_mask)

In [ ]:
import geemap


# Create a map
Map = geemap.Map(center=[-1.37, 38.01], zoom=8)  # Centered on Kitui, Kenya (example coords)

# Define visualization parameters
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['white', 'green']  # Adjust based on your mask values
}

vis_NDVIparams = {
    'min': 0,
    'max': 10000,

}

# Add the image layer
Map.addLayer(crop_mask, vis_params, "Masked Crop Image")

Map.addLayer(datamasked.select('NDVI'), vis_NDVIparams, "NDVI Image")
Map.addLayer(data_filled_masked.select('NDVI'), vis_NDVIparams, "NDVI Image filled")
Map.addLayer(data_filled_SG_masked.select('NDVI_smooth'), vis_NDVIparams, "NDVI Image filled + SG")

# Display the map
Map

## Download data for a specific administrative level

In [ ]:
data_downloader = GEEMODIS(configuration['DATA_DOWNLOAD']['ADM0_NAME'], configuration['DATA_DOWNLOAD']['product'])
data_downloader.initialize_query(configuration['DATA_DOWNLOAD']['starting_date'], configuration['DATA_DOWNLOAD']['ending_date'])

feature_name = 'Kericho'

not_smootheddf = data_downloader.get_adm_timeseries(adm_level='ADM1', feature_name= feature_name, fill_gaps=True, sg = False)
smootheddf3 = data_downloader.get_adm_timeseries(adm_level='ADM1', feature_name= feature_name, fill_gaps=True, sg = True, window_size = 3)
smootheddf5 = data_downloader.get_adm_timeseries(adm_level='ADM1', feature_name= feature_name, fill_gaps=True, sg = True, window_size = 5)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(not_smootheddf['date'], not_smootheddf[not_smootheddf.columns[1]], marker='o', linestyle='-', label = 'Raw')
plt.plot(smootheddf3['date'], smootheddf3[smootheddf3.columns[1]], marker='o', linestyle='-', c = 'red', label = 'Smoothed window 3')
plt.plot(smootheddf5['date'], smootheddf5[smootheddf5.columns[1]], marker='o', linestyle='-', c = 'green', label = 'Smoothed window 5')

plt.title(f"{not_smootheddf.columns[1]} Time Series for {feature_name}")
plt.xlabel("Date")
plt.ylabel(not_smootheddf.columns[1])
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
smootheddf5